In [8]:
from langchain_openai.chat_models import ChatOpenAI
from langchain_community.chat_models.ollama import ChatOllama
from langchain_core.callbacks import StreamingStdOutCallbackHandler
from langchain.schema.runnable import RunnablePassthrough
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain.memory import ConversationBufferMemory
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

# 챗 지피티
chat = ChatOpenAI(
    temperature=0.1,
    model="gpt-3.5-turbo",
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

# 로컬 LLM (ollama)
chat_ollama = ChatOllama(
    temperature=0.1,
    model="openhermes:latest",
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

memory = ConversationBufferMemory(return_messages=True)


def load_memory(_):
    x = memory.load_memory_variables({})
    return {"history": x["history"]}


example = [
    {
        "영화제목": "탑건",
        "answer": "🛩️👨‍✈️🔥",
    },
    {
        "영화제목": "대부",
        "answer": "👨‍👨‍👦🔫🍝",
    },
]

output_parser = StrOutputParser()

example_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "human",
            "영화 제목을 말하면 영화를 설명하는 이모티콘으로 답변해줘. 영화제목은 {영화제목}야",
        ),
        ("ai", "{answer}"),
    ]
)

example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=example,
)


explain_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "너는 영화에 대해 이모티콘 3개로 설명할 수 있어."),
        example_prompt,
        MessagesPlaceholder(variable_name="history"),
        ("human", "{영화제목}"),
    ]
)

# 체인 생성 (중간에 chat_ollama를 넣어서 로컬 LLM을 사용함 -> chat으로 바꾸면 오픈AI 사용)
chain = (
    RunnablePassthrough.assign(history=load_memory)
    | explain_prompt
    | chat_ollama
    | output_parser
)

chain2 = RunnablePassthrough.assign(history=load_memory) | chat_ollama | output_parser
# 입력한 내용에 대해서 영화 정보를 알려줌 (Input 사용)
explian_first = chain.invoke({"영화제목": input()})

🚢⛵️🌊💔탑건제 영화 "타이타닉"

'탑건제 영화 "타이타닉"'

In [ ]:
chain.invoke("")